# Introduction


This notebook has been prepared as a demo on how to perform aperture photometry in SBC images that contain an elevated dark rate. This problem arises when the detector temperature goes above ~25C. Keep [ISR ACS 2017-04](http://www.stsci.edu/hst/acs/documents/isrs/isr1704.pdf) open in a separate browser tab since it will be referred to in this document. The elevated dark primarily affects a large region of the detector just above and to the right of center (see Figure 4 of ISR). 

The following cell loads all the python packages needed for this example. 

In [ ]:
from astroquery.mast import Observations
from drizzlepac.astrodrizzle import AstroDrizzle as adriz
from matplotlib.colors import LogNorm
import glob
from astropy.table import Table
from astropy.io import fits
import math
import numpy as np
import shutil

# Data

The necessary data needs to be downloaded and placed in the correct locations.

To run this notebook you will need the RAW dark frames from the calibration program 13961 and FLT and ASN files from visit 11 of program 13655. Place those images in the same directory as this notebook.

The following commands query MAST for the necessery products and then download them.

In [ ]:
science_list = Observations.query_criteria(obs_id='JCMC11*')
Observations.download_products(science_list['obsid'],mrp_only=False,download_dir='./science',
                               productSubGroupDescription=['FLT','ASN'])

darks_list = Observations.query_criteria(proposal_id='13961',obstype='cal')
Observations.download_products(darks_list['obsid'],mrp_only=False,download_dir='./darks',
                               productSubGroupDescription=['RAW'])

In [ ]:
dark_files = glob.glob('darks/mastDownload/HST/*/*fits')
for im in dark_files:
    shutil.move(im,'darks/')
shutil.rmtree('darks/mastDownload/')

science_files = glob.glob('science/mastDownload/HST/*/*fits')
for im in science_files:
    shutil.move(im,'./')
shutil.rmtree('science/')

# Overview


What we will try to do here is use dark frames to make a drizzled product that mimics the background in the science product. We will use that drizzled dark product to measure the dark rate to be subtracted from the photometry of the science product.

We are reading in the log file and printing its content. Columns **mdectodt1** and **mdecodt2** contain the temperature at the beginning and end of the exposure respectively.

In [ ]:
obslog = Table.read('data.log',format='ascii.fixed_width')
obslog

We will create a new column called **avgtemp**, containing the average temperature, and add it to the table.

In [ ]:
obslog['avgtemp'] = (obslog['mdecodt1']+obslog['mdecodt2'])/2.
obslog

# Creating dark images


The table above shows that the second image taken with the F165LP filter has an average teperature above 25C and therefore has been affected by elevated dark rate. Specifically, the elevated dark rate comes from the final image in the visit **jcmc11e6q_flt.fits**, which has a temperature above 25C. 

Let's make drizzled products for each filter. We do this by making a list of the ASN files and drizzling those one by one. The drizzle parameters below are the appropriate ones for SBC data. In particular, Steps 1-6 of the drizzling procedure have been turned off since their purpose is to identify and mask cosmic rays. SBC images do not have cosmic rays so you can just proceed straight to the drizzling. We specify that the drizzle products have a plate scale of 0.025"/pixel.

In [ ]:
asnlist = glob.glob('*asn.fits')

for asn in asnlist:
    
    exps = fits.getdata(asn)
    first_im = exps['memname'][0]
    output_name = fits.getval(first_im.lower()+'_flt.fits','filter1',ext=0)
    
    adriz(asn,
          output=output_name,
          runfile='',
          context=False,
          build=False,
          preserve=False,
          clean=True,
          static=False,
          skysub=False,
          driz_separate=False,
          median=False,
          blot=False,
          driz_cr=False,
          driz_combine=True,
          final_wcs=True,
          final_scale=0.025
         )

Figure 1 of the ISR shows that the dark rate above 25C varies. We therefore can't simply use the dark frames that have the same temperature as your affected science frame. We need to find the dark frame that contains a *dark rate* similar to your affected image. Below we open the two F165LP science frames and print out the sum of the pixels inside of a 200x200 box centered at (750,680). We then do the same for all the dark frames.

In [ ]:
darklist = glob.glob('darks/*fits')
print('jcmc11ctq_flt.fits',np.sum(fits.getdata('jcmc11ctq_flt.fits')[650:850,580:780]))
print('jcmc11e6q_flt.fits',np.sum(fits.getdata('jcmc11e6q_flt.fits')[650:850,580:780]))

for dark in darklist:
    
    sci = fits.getdata(dark)
    print(dark[-18:],np.sum(sci[650:850,580:780]))

It looks like the two dark frames that come closest to the science frames are **jcrx01iqq** and **jcrx01iyq**. We will use those to make a combined master dark frame.

To do this, we will insert the data of the dark images into copies of the science images. We will do this so that we keep the WCS information in the header to make a drizzled product. We must remember to adjust the exposure time of the copies of the science frames to that of the dark frames so that the drizzled products have the correct count rates.

In [ ]:
!cp jcmc11ctq_flt.fits dark1.fits
!cp jcmc11e6q_flt.fits dark2.fits

darkdat1 = fits.getdata('darks/jcrx01iiq_raw.fits').astype(np.float)
with fits.open('dark1.fits',mode='update') as hdu1:
    hdu1[1].data[:,:] = darkdat1
    hdu1[0].header['exptime'] = fits.getval('darks/jcrx01iiq_raw.fits','exptime',ext=0)
    
darkdat2 = fits.getdata('darks/jcrx01iyq_raw.fits').astype(np.float)
with fits.open('dark2.fits',mode='update') as hdu2:
    hdu2[1].data[:,:] = darkdat2
    hdu2[0].header['exptime'] = fits.getval('darks/jcrx01iyq_raw.fits','exptime',ext=0)

We can now make the drizzled dark frame using the two individual dark frames we just made as inputs. The drizzle parameters are the same as the ones used to make the science drizzled products.

In [ ]:
adriz(['dark1.fits','dark2.fits'],
        output='masterdark',
        runfile='',
        context=False,
        build=False,
        preserve=False,
        clean=True,
        static=False,
        skysub=False,
        driz_separate=False,
        median=False,
        blot=False,
        driz_cr=False,
        driz_combine=True,
        final_wcs=True,
        final_scale=0.025
       )

We will now display the images to confirm that they show similar elevated dark rates. You might want do display them in DS9 (or any other viewer) outside of this notebook so you can play with the stretch a bit and so you can see bigger versions of the images.

In [ ]:
f165lp = fits.getdata('F165LP_drz_sci.fits')
masterdark = fits.getdata('masterdark_drz_sci.fits')

fig,ax = plt.subplots(1,2,figsize=(16,12))

ax[0].imshow(f165lp,norm=LogNorm(),vmin=1e-5,vmax=0.01,interpolation='nearest',cmap='plasma')
ax[1].imshow(masterdark,norm=LogNorm(),vmin=1e-5,vmax=0.01,interpolation='nearest',cmap='plasma')

# Photometry


The images look comparable. We will now proceed to performing some photometric analysis to estimate the dark current in the source.

First we import some tasks from the `photutils` package and then set up two apertures we will use to measure the flux of different regions in the images.  

In [ ]:
from photutils import EllipticalAperture,aperture_photometry

#This line creates two elliptical apertures.
aper = EllipticalAperture([(735,710),(200,200)],a=70,b=40,theta=0.5*math.pi)

I am overplotting the two apertures in the images so you can see their locations.

In [ ]:
fig,ax = plt.subplots(1,2,figsize=(16,12))

ax[0].imshow(f165lp,norm=LogNorm(),vmin=1e-5,vmax=0.001,interpolation='nearest',cmap='plasma')
ax[1].imshow(masterdark,norm=LogNorm(),vmin=1e-5,vmax=0.001,interpolation='nearest',cmap='plasma')
aper.plot(ax=ax[0],edgecolor='blue')
aper.plot(ax=ax[1],edgecolor='blue')

Finally, we do the photometry using the two apertures on both images. We print out the tables to see the results.

In [ ]:
f165lp_phot = aperture_photometry(f165lp, aper)
masterdark_phot = aperture_photometry(masterdark, aper)

In [ ]:
print(f165lp_phot)

In [ ]:
print(masterdark_phot)

In [ ]:
f165lp_phot['aperture_sum']-masterdark_phot['aperture_sum']

The target aperture has 2.86 cts/sec, while the same aperture in the dark frame has 0.322 cts/sec. That means that ~11% of the flux in your source comes from dark current and should be subtracted out, leaving a flux for you source of 2.543 cts/sec. 

# Final thoughts


1. The difference in flux in the second aperture (the one in the lower left portion of the image) shows that there is a small residual background of ~0.02 cts/sec in the science frame. This could be real background from the sky (and not dark current from the detector that you might want to account for properly in your flux and error budget.

2. The dark frame we created does not have the exact same dark count rate as we measured in the science frame. You could try searching for other darks that more closely resemble your science frame. 

3. These problems can be avoided using a few strategies detailed in [ISR ACS 2018-??](http://www.stsci.edu/hst/acs/documents/isrs/)